In [36]:
import numpy
import sklearn.svm
#from GA import GA

In [37]:
import scipy.io as sio

In [38]:
Data=sio.loadmat('./intermediate/Feature_vectors.mat')

In [39]:
X=Data['Features_vectors']

In [40]:
X = X.astype(float)

In [41]:
import numpy as np
X=np.transpose(X)

In [42]:
X = X[~np.all(X == 0, axis=1)]

In [43]:
X=np.transpose(X)

In [44]:
labels=sio.loadmat('./labels77.mat')

In [45]:
y=labels['labels']

In [46]:
y=y[:, 0]

In [47]:
n_samples, n_features = X.shape    # number of samples and number of features

In [48]:
from sklearn.model_selection import KFold
from sklearn import svm
from sklearn.metrics import accuracy_score

In [49]:
# split data into 5 folds
kf5=KFold(n_splits=5,shuffle=True)

In [50]:
num_fea = [i for i in range(10,110,10)]
clf = svm.LinearSVC()    # linear SVM
correct=0
accuracy=[]

In [51]:
X.shape

(77, 589)

In [52]:
from sklearn.feature_selection import RFE


In [53]:
rfe = RFE(estimator=clf, n_features_to_select=5, step=1)
#rfe = rfe.fit(x_train, y_train)

In [ ]:
accuracy=[]
featureranking=[]
correct=0
for train_index, test_index in kf5.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    rfe = RFE(estimator=clf, n_features_to_select=5, step=1)
    rfe = rfe.fit(X_train, y_train)
    #print('Chosen best 5 feature by rfe:',X_train.columns[rfe.support_])
    print(X_train[[rfe.support_]])
    
    #selected_features = X[:, idx[0:k]]
    #featureranking.extend([idx])
    # train a classification model with the selected features on the training dataset
    #clf.fit(selected_features[train_index], y[train_index])  # predict the class labels of test data
    #y_predict = clf.predict(selected_features[test_index])
    # obtain the classification accuracy on the test data
    #acc = accuracy_score(y[test_index], y_predict)
    #correct = correct + acc
    # output the average classification accuracy over all folds
#accuracy=float(correct)/cv.get_n_splits(X)

In [73]:
from sklearn.feature_selection import RFECV
rfecv = RFECV(estimator=clf, step=1, cv=5,scoring='accuracy')   #5-fold cross-validation
rfecv = rfecv.fit(X, y)

In [77]:
rfecv.grid_scores_[2]

0.6776470588235294

In [61]:
def rfecv_FS(cv,X_train,y_train):
    rfecv = RFECV(estimator=clf, step=1, cv=cv.get_n_splits(X),scoring='accuracy')   #5-fold cross-validation
    rfecv = rfecv.fit(X_train, y_train)
    idx=rfecv.ranking_
    return(idx)

In [59]:
featureranking=[]

In [68]:
for train_index, test_index in kf5.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    idx=rfecv_FS(kf5,X_train,y_train)
    selected_features = X[:, idx[0:10]]
    featureranking.extend([idx])

    # train a classification model with the selected features on the training dataset
    clf.fit(selected_features[train_index], y[train_index])  # predict the class labels of test data
    y_predict = clf.predict(selected_features[test_index])
    # obtain the classification accuracy on the test data
    acc = accuracy_score(y[test_index], y_predict)
    correct = correct + acc

    # output the average classification accuracy over all folds
accuracy=float(correct)/kf5.get_n_splits(X)
    #return(np.array(featureranking),accuracy)
    #print('Optimal number of features :', rfecv.n_features_)
    #print('Best features :', X_train[[rfecv.support_]])

In [69]:
matrice=np.array(featureranking)

In [72]:
accuracy

0.6266666666666667

In [71]:
matrice.shape

(11, 589)

### SFS backward

In [ ]:
from mlxtend.feature_selection import SequentialFeatureSelector

feature_selector = SequentialFeatureSelector(clf,
           k_features=589,
           forward=True,
           verbose=2,
           scoring='roc_auc',
           cv=4)

### FS via gridsearch

In [ ]:
from itertools import combinations

all_comb = []
for size in range(1, 5):
    all_comb += list(combinations(range(X.shape[1]), r=size))
print(all_comb)

In [ ]:
from mlxtend.feature_selection import ColumnSelector
#from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline

pipe = make_pipeline(StandardScaler(),
                     ColumnSelector(),
                     clf)
# Optimal parameters for SVM using gridsearch
param_grid = {'columnselector__cols': all_comb}
#param_grid = {'C':[1,10,100,1000],'gamma':[1,0.1,0.001,0.0001], 'kernel':['linear','rbf']}

grid = GridSearchCV(clf,param_grid,cv=5,n_jobs=-1)
grid.fit(X, y)
#print('Best parameters:', grid.best_params_)
#print('Best performance:', grid.best_score_)

In [ ]:
def training(cv,k,FS_method):
    accuracy=[]
    featureranking=[]
    correct=0
    for train_index, test_index in cv.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        if FS_method==reliefF:
            idx=GA_FS(X_train,y_train)
  
        selected_features = X[:, idx[0:k]]
        featureranking.extend([idx])

        # train a classification model with the selected features on the training dataset
        clf.fit(selected_features[train_index], y[train_index])  # predict the class labels of test data
        y_predict = clf.predict(selected_features[test_index])
        # obtain the classification accuracy on the test data
        acc = accuracy_score(y[test_index], y_predict)
        correct = correct + acc

        # output the average classification accuracy over all folds
    accuracy=float(correct)/cv.get_n_splits(X)
    return(np.array(featureranking),accuracy)

============Main==============================